In [16]:
'''
    MNIST with dropout by PyTorch
'''

'\n    MNIST with dropout by PyTorch\n'

In [17]:
# Lab 10 MNIST and softmax
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [18]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [19]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
drop_prob = 0.3

In [20]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [21]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [22]:
# set layer => front number = input / back number = output (size of weights matrix)
linear1 = torch.nn.Linear(784, 512, bias=True)
linear2 = torch.nn.Linear(512, 512, bias=True)
linear3 = torch.nn.Linear(512, 512, bias=True)
linear4 = torch.nn.Linear(512, 512, bias=True)
linear5 = torch.nn.Linear(512, 10, bias=True)

# activation function
relu = torch.nn.ReLU()

# dropout with ratio 0.5 => will use 50% of whole nodes (=unit or neuron or whatever)
dropout = torch.nn.Dropout(p=drop_prob)

'''
    Sequential:
        linear1 => ReLU => linear2 => ReLU => linear3 => ReLU => linear4 => ReLU => linear5 => Softmax
'''

'\n    Sequential:\n        linear1 => ReLU => linear2 => ReLU => linear3 => ReLU => linear4 => ReLU => linear5 => Softmax\n'

In [23]:
# xavier init of weights
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.xavier_uniform_(linear4.weight)
torch.nn.init.xavier_uniform_(linear5.weight)

Parameter containing:
tensor([[-0.0565,  0.0423, -0.0155,  ...,  0.1012,  0.0459, -0.0191],
        [ 0.0772,  0.0452, -0.0638,  ...,  0.0476, -0.0638,  0.0528],
        [ 0.0311, -0.1023, -0.0701,  ...,  0.0412, -0.1004,  0.0738],
        ...,
        [ 0.0334,  0.0187, -0.1021,  ...,  0.0280, -0.0583, -0.1018],
        [-0.0506, -0.0939, -0.0467,  ..., -0.0554, -0.0325,  0.0640],
        [-0.0183, -0.0123,  0.1025,  ..., -0.0214,  0.0220, -0.0741]],
       requires_grad=True)

In [24]:
# set model
model = torch.nn.Sequential(
    linear1, relu, dropout,
    linear2, relu, dropout,
    linear3, relu, dropout,
    linear4, relu, dropout,
    linear5
).to(device)

In [25]:
# set cost and optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [26]:
# train with dropout
total_batch = len(data_loader)

# will use dropout
model.train()
for epoch in range(training_epochs):
    # mean of cost each epoch(=learning)
    avg_cost = 0
    
    for X, Y in data_loader:
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
    
        cost = criterion(hypothesis, Y)
        
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
    
    print('Epoch: {:2d}/{}, Cost: {:.9f}'.format(epoch, training_epochs, avg_cost.item()))
    
print('Train Finished')

Epoch:  0/15, Cost: 0.312231481
Epoch:  1/15, Cost: 0.141670987
Epoch:  2/15, Cost: 0.112299174
Epoch:  3/15, Cost: 0.095792152
Epoch:  4/15, Cost: 0.082006089
Epoch:  5/15, Cost: 0.073913589
Epoch:  6/15, Cost: 0.067826346
Epoch:  7/15, Cost: 0.066471420
Epoch:  8/15, Cost: 0.056462843
Epoch:  9/15, Cost: 0.054717381
Epoch: 10/15, Cost: 0.050787512
Epoch: 11/15, Cost: 0.048931323
Epoch: 12/15, Cost: 0.046048012
Epoch: 13/15, Cost: 0.045115236
Epoch: 14/15, Cost: 0.042753655
Train Finished


In [27]:
# test without dropout (dropout only for train procedure)
with torch.no_grad():
    
    # will not use dropout
    model.eval()

    # Test the model using test sets
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

/opt/conda/lib/python3.6/site-packages/torchvision/datasets/mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/opt/conda/lib/python3.6/site-packages/torchvision/datasets/mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9824000000953674
Label:  8
Prediction:  8
